모델 학습

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch

In [ ]:
file_path = '/kaggle/input/new-qa-data/QA_data_new.csv'
df = pd.read_csv(file_path)

In [ ]:
def text_preprocessing(text):
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.lower()
    return text

In [ ]:
df['question'] = df['question'].apply(text_preprocessing)

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'])

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [ ]:
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')
model = BertForSequenceClassification.from_pretrained('kykim/bert-kor-base', num_labels=2)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['question'], padding='max_length', truncation=True)

In [ ]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=10,
    report_to='none'
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_datasets['train'],
    eval_dataset=encoded_datasets['validation']
)

In [ ]:
trainer.train()

학습된 모델 불러오기

In [3]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch

2024-07-23 00:06:16.918473: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 00:06:16.918619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 00:06:17.052994: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
load_tokenizer = BertTokenizer.from_pretrained('/kaggle/input/bert-kor-base-lawqa/bert-kor-base-lawQA')
load_model = BertForSequenceClassification.from_pretrained('/kaggle/input/bert-kor-base-lawqa/bert-kor-base-lawQA')

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
load_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [7]:
def predict(query):
    inputs = load_tokenizer(query, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = load_model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return "법률질문" if predictions.item() == 1 else "비법률질문"

In [26]:
#query1 = "점심에 마라탕 먹으려는데 괜찮은곳 있어?"
#query1 = '어제 교통사고를 당했는데 상대방이 후속 조치를 하지 않고 도망갔어.'
query1 = '''
청구취지
원고(피해자): 김철수
피고(가해자): 박영희
사건 번호: 2024형12345

청구취지:
피고는 원고에게 5,000만 원을 배상하라.
피고는 원고에게 위 금액에 대하여 이 판결이 확정된 다음날부터 다 갚는 날까지 연 5%의 비율로 계산한 금원을 지급하라.
소송 비용은 피고의 부담으로 한다.

청구원인:
피고 박영희는 2024년 5월 15일 서울특별시 강남구에서 원고 김철수를 폭행하여 원고에게 약 4주의 치료를 요하는 상해를 입혔습니다. 이로 인해 원고는 신체적 고통과 정신적 고통을 겪었고, 병원 치료비와 기타 손해가 발생하였습니다.
피고의 행위는 형법 제257조 제1항의 상해죄에 해당합니다. 따라서 피고는 민법 제750조에 따라 원고에게 손해배상 의무를 집니다.
원고는 피고의 폭행으로 인한 상해로 인하여 병원 치료비 2,000만 원, 정신적 고통에 대한 위자료 3,000만 원을 합산하여 총 5,000만 원을 청구합니다.

증거자료:

사건 발생 당시 CCTV 영상
경찰 수사 기록
병원 진단서 및 치료 기록
피해자의 진술서'''

query2 = '''
조리 방법
패티 준비

큰 그릇에 다진 소고기를 넣고 소금과 후추를 넣어 잘 섞습니다.
소고기 혼합물을 4등분하여 각각 약 125g씩 나눕니다.
각 부분을 손으로 둥글게 만든 다음, 납작하게 눌러 패티 모양을 만듭니다.
패티 굽기

중간 높은 불로 그릴이나 프라이팬을 예열합니다.
패티를 예열된 그릴이나 프라이팬에 올려 한쪽 면을 약 3-4분간 굽습니다. (중간에 뒤집지 말고 그대로 두세요)
뒤집어서 다른 쪽 면도 3-4분간 굽습니다. (패티의 두께와 원하는 굽기 정도에 따라 시간 조절)
패티가 거의 다 익으면, 치즈 한 장을 패티 위에 올려 녹입니다.
번 준비

햄버거 번을 반으로 잘라 그릴이나 프라이팬에서 살짝 굽습니다. (약 1-2분)
햄버거 조립

굽은 햄버거 번 아래쪽에 케첩과 머스터드를 바릅니다.
상추 한 장을 올립니다.
치즈가 녹은 패티를 상추 위에 올립니다.
슬라이스한 토마토와 양파를 올립니다.
원한다면 피클을 추가합니다.
번 위쪽을 덮어 햄버거를 완성합니다.
'''


query3 = '''
청구취지:
피고는 원고에게 5,000만 원을 배상하라.
피고는 원고에게 위 금액에 대하여 이 판결이 확정된 다음날부터 다 갚는 날까지 연 5%의 비율로 계산한 금원을 지급하라.
소송 비용은 피고의 부담으로 한다.

청구원인:
피고 박영희는 2024년 5월 15일 서울특별시 강남구에서 원고 김철수를 폭행하여 원고에게 약 4주의 치료를 요하는 상해를 입혔습니다. 이로 인해 원고는 신체적 고통과 정신적 고통을 겪었고, 병원 치료비와 기타 손해가 발생하였습니다.
피고의 행위는 형법 제257조 제1항의 상해죄에 해당합니다. 따라서 피고는 민법 제750조에 따라 원고에게 손해배상 의무를 집니다.
원고는 피고의 폭행으로 인한 상해로 인하여 병원 치료비 2,000만 원, 정신적 고통에 대한 위자료 3,000만 원을 합산하여 총 5,000만 원을 청구합니다.

증거자료:

사건 발생 당시 CCTV 영상
경찰 수사 기록
병원 진단서 및 치료 기록
피해자의 진술서
'''

query4 = '''
작업 방법
준비 단계

큰 박스에 다양한 색상의 종이를 넣고 무작위로 섞습니다.
종이 혼합물을 4등분하여 각각 약 30장씩 나눕니다.
각 부분을 손으로 둥글게 만든 다음, 납작하게 눌러 모양을 만듭니다.
조립 단계

중간 높은 온도로 전구를 예열합니다.
준비된 종이 뭉치를 예열된 전구 근처에 올려 약 3-4분간 유지합니다. (중간에 만지지 말고 그대로 두세요)
뒤집어서 다른 쪽 면도 3-4분간 유지합니다. (종이의 두께와 원하는 모양에 따라 시간 조절)
종이가 거의 다 형성되면, 스티커 한 장을 종이 위에 올려 부착합니다.
마무리 단계

상자를 반으로 잘라 접착제로 살짝 고정합니다. (약 1-2분)
조립된 종이 뭉치를 상자 아래쪽에 놓습니다.
스티커가 부착된 종이를 상자 위에 올립니다.
슬라이스한 색종이와 리본을 올립니다.
원한다면, 추가 장식을 더합니다.
상자 위쪽을 덮어 작업을 완성합니다.

'''

In [27]:
print(predict(query1))
print(predict(query2))
print(predict(query3))
print(predict(query4))

법률질문
비법률질문
법률질문
비법률질문
